In [1]:

import numpy as np
import angular_momentum
import wigner_d
import cg
import math
import random
from wigner_d import WignerD
from cg import ClebschGordanCoefficients
from IPython.display import display


# Angular momentum operators
The angular momentum operators $J_x^{(j)}$, $J_y^{(j)}$, $J_z^{(j)}$, $J_+^{(j)}$, and $J_-^{(j)}$ in the bases $|j, m \rangle$ given a $j \in \{ n / 2 \mid n \in \mathbb{N} \}$. We will use the natural unit such that $\hbar = 1$.

In [2]:
j = 1/2
angular_momentum.J_x(j), angular_momentum.J_y(j), angular_momentum.J_z(j)


(array([[0. , 0.5],
        [0.5, 0. ]]),
 array([[0.+0.j , 0.+0.5j],
        [0.-0.5j, 0.+0.j ]]),
 array([[1., 0.],
        [0., 1.]]))

check that $\left( j_y^{(j=1)} \right)^3 = J_y^{(j=1)}$


In [3]:
j = 1
np.linalg.matrix_power(angular_momentum.J_y(j), 3) - angular_momentum.J_y(j)  # should be zero up to machine precision


array([[0.+0.00000000e+00j, 0.+1.11022302e-16j, 0.+0.00000000e+00j],
       [0.-1.11022302e-16j, 0.+0.00000000e+00j, 0.+1.11022302e-16j],
       [0.+0.00000000e+00j, 0.-1.11022302e-16j, 0.+0.00000000e+00j]])

In [4]:
angular_momentum.J_plus(1), angular_momentum.J_minus(1)

(array([[0.        , 0.        , 0.        ],
        [1.41421356, 0.        , 0.        ],
        [0.        , 1.41421356, 0.        ]]),
 array([[0.        , 1.41421356, 0.        ],
        [0.        , 0.        , 1.41421356],
        [0.        , 0.        , 0.        ]]))

# Wigner D matrices

In [5]:
# spin-1/2
# gives -I after a full rotation
j = 1 / 2
beta = 2 * np.pi
wigner_d.wigner_d(j, beta).round(12)


array([[-1.,  0.],
       [-0., -1.]])

In [19]:
j = 1
beta = 2 * np.pi
wigner_d.wigner_d(j, beta).round(12)


array([[ 1., -0.,  0.],
       [ 0.,  1., -0.],
       [ 0.,  0.,  1.]])

In [7]:
j = 1
alpha = random.uniform(0, 2 * math.pi)
beta = random.uniform(0, math.pi)
gamma = random.uniform(0, 2 * math.pi)
d = wigner_d.wigner_d(j, beta)
# given a d, we can calculate the D matrix
wigner_d.wigner_D_from_d(d, alpha, gamma) - wigner_d.wigner_D(j, alpha, beta, gamma)


array([[0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j],
       [0.+0.j, 0.+0.j, 0.+0.j]])

In [8]:
alpha = np.random.rand(1) * 2 * np.pi
beta = np.random.rand(1) * np.pi
gamma = np.random.rand(1) * 2 * np.pi
wd = WignerD(jmax=2, alpha=alpha, beta=beta, gamma=gamma)
wd

WignerDDict(jmax=2, alpha=[2.43583087], beta=[2.3421765], gamma=[2.59523603])

In [9]:
wd.set_jmax(3)

In [10]:
wd.wigner_D(1)

array([[ 0.04744702-0.1438125j , -0.38585594+0.32882103j,
         0.83780452-0.13469316j],
       [ 0.43315808-0.26340492j, -0.69712541+0.j        ,
        -0.43315808-0.26340492j],
       [ 0.83780452+0.13469316j,  0.38585594+0.32882103j,
         0.04744702+0.1438125j ]])

In [11]:
# a dictionary of all stored little d matrices
wd.d

{0.0: array([[1.]]),
 0.5: array([[ 0.38914945,  0.92117463],
        [-0.92117463,  0.38914945]]),
 1.0: array([[ 0.15143729,  0.50695964,  0.84856271],
        [-0.50695964, -0.69712541,  0.50695964],
        [ 0.84856271, -0.50695964,  0.15143729]]),
 1.5: array([[ 0.05893174,  0.24162142,  0.57195385,  0.78167444],
        [-0.24162142, -0.60150368, -0.50267405,  0.57195385],
        [ 0.57195385,  0.50267405, -0.60150368,  0.24162142],
        [-0.78167444,  0.57195385, -0.24162142,  0.05893174]]),
 2.0: array([[ 0.02293325,  0.10857285,  0.31476933,  0.60837636,  0.72005867],
        [-0.10857285, -0.36257886, -0.61213178, -0.33454655,  0.60837636],
        [ 0.31476933,  0.61213178,  0.22897576, -0.61213178,  0.31476933],
        [-0.60837636, -0.33454655,  0.61213178, -0.36257886,  0.10857285],
        [ 0.72005867, -0.60837636,  0.31476933, -0.10857285,  0.02293325]]),
 2.5: array([[ 0.00892446,  0.04723812,  0.15813689,  0.37433355,  0.62656958,
          0.66329978],
       

# Clebsch-Gordan coefficients
The CG coefficient $\langle j_1, j_2; m_1, m_2 | j_1, j_2; j, m \rangle$

In [12]:
# compute a single Clebsch-Gordan coefficient
cg.cg_coefficient(
    j1=1/2, j2=1/2,
    m=0, j=1,
    m1=-1/2, m2=1/2
)

0.7071067811865476

In [13]:
# display a table of Clebsch-Gordan coefficients given j1, j2, and m
# should be empty: https://en.wikipedia.org/wiki/Table_of_Clebsch%E2%80%93Gordan_coefficients
cg.cg_table(j1=1, j2=0, m=1/2)


j,1.0
"(m1, m2)",


In [14]:
# https://en.wikipedia.org/wiki/Table_of_Clebsch%E2%80%93Gordan_coefficients
cg.cg_table(j1=1, j2=1/2, m=1/2)

j,0.5,1.5
"(m1, m2)",,
"(0.0, 0.5)",-0.577350,0.816497
"(1.0, -0.5)",0.816497,0.577350


In [15]:
# CG table with all possible m's
# returns a dict of pd.DataFrame
_ = cg.cg_tables_all_m(1/2, 1/2, display_tables=True)

j1 = 0.5, j2 = 0.5
m = -1.0:


j,1.0
"(m1, m2)",
"(-0.5, -0.5)",1.0


m = 0.0:


j,0.0,1.0
"(m1, m2)",,
"(-0.5, 0.5)",-0.707107,0.707107
"(0.5, -0.5)",0.707107,0.707107


m = 1.0:


j,1.0
"(m1, m2)",
"(0.5, 0.5)",1.0


In [16]:
# stores all CG coefficients up to jmax=2
cg_table = ClebschGordanCoefficients(jmax=2)
# get the pd.DataFrame for the CG coefficients for j1 = j2 = 1/2 and m = 1
cg_table[0.5, 0.5, 1]


j,1.0
"(m1, m2)",
"(0.5, 0.5)",1.0


In [17]:
# displays the computed CG table with j1 = j2 = 1
for t in cg_table[1, 1].items():
    print(f'm = {t[0]}:')
    display(t[1])


m = -2.0:


j,2.0
"(m1, m2)",
"(-1.0, -1.0)",1.0


m = -1.0:


j,1.0,2.0
"(m1, m2)",,
"(-1.0, 0.0)",-0.707107,0.707107
"(0.0, -1.0)",0.707107,0.707107


m = 0.0:


j,0.0,1.0,2.0
"(m1, m2)",,,
"(-1.0, 1.0)",0.57735,-0.707107,0.408248
"(0.0, 0.0)",-0.57735,0.000000,0.816497
"(1.0, -1.0)",0.57735,0.707107,0.408248


m = 1.0:


j,1.0,2.0
"(m1, m2)",,
"(0.0, 1.0)",-0.707107,0.707107
"(1.0, 0.0)",0.707107,0.707107


m = 2.0:


j,2.0
"(m1, m2)",
"(1.0, 1.0)",1.0


In [18]:
# displays the computed CG table of with j1=3/2 and j2=1/2 with all possible m
# returns a dictionary of CG tables (pd.DataFrame)
_ = cg.cg_tables_all_m(j1=3/2, j2=1/2, display_tables=True)


j1 = 1.5, j2 = 0.5
m = -2.0:


j,2.0
"(m1, m2)",
"(-1.5, -0.5)",1.0


m = -1.0:


j,1.0,2.0
"(m1, m2)",,
"(-1.5, 0.5)",-0.866025,0.500000
"(-0.5, -0.5)",0.500000,0.866025


m = 0.0:


j,0.0,1.0,2.0
"(m1, m2)",,,
"(-0.5, 0.5)",0,-0.707107,0.707107
"(0.5, -0.5)",0,0.707107,0.707107


m = 1.0:


j,1.0,2.0
"(m1, m2)",,
"(0.5, 0.5)",-0.500000,0.866025
"(1.5, -0.5)",0.866025,0.500000


m = 2.0:


j,2.0
"(m1, m2)",
"(1.5, 0.5)",1.0
